In [ ]:
import numpy as np


In [ ]:
# Number of tasks and resources
N = 10
M = 5

# Chromosome length
chromosome_length = N

# Generate a random chromosome using NumPy
chromosome = np.random.randint(1, M+1, chromosome_length)

# Display the generated chromosome
print("Generated Chromosome:", chromosome)
